In [ ]:
import warnings,gensim
warnings.filterwarnings('ignore')
model_path = 'GoogleNews-vectors-negative300.bin.gz'
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path,
binary=True) # Loading the model using genism
import json
import h5py
import numpy as np
import copy
from random import shuffle, seed
import sys
import os.path
import argparse
import glob
import numpy as np
import scipy.io
import pdb
import string
import h5py
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import gensim
import json
import re
import cv2
import matplotlib.pyplot as plt

In [ ]:
def extract_feat(doc):
    feat = []
    for word in doc:
    try:
    feat.append(model_w2v[word])
    except:
    pass
 return feat

In [ ]:
def tokenize(sentence):
    return [i for i in re.split(r"([-.\"',:? !\$#@~()*&\^%;\[\]/\\\+<>\n=])", sentence) if i!=''
and i!=' ' and i!='\n'];
def prepro_question(imgs, method):
 # preprocess all the question
 print('example processed tokens:')

 Input: The question from trainset.json
 Performs tokenization and lowering of the question
 Output: Embedded version of the question
 Note that: We have still not padded the questions, just for information each of the word will be a 300 Dimensional Vector,
 Hence, the word vector (which will be obtained after padding) will be a (21,300) Dimensional vector.

In [ ]:
for i,img in enumerate(imgs):
    s = img['question'].lower()
    if method == 'nltk':
        txt = word_tokenize(str(s).lower())
    else:
        txt = tokenize(s)
    img['processed_tokens'] = txt
    if i < 10: print(txt)
        if i % 1000 == 0:
            sys.stdout.write("processing %d/%d (%.2f%% done) \r" % (i, len(img),
        i*100.0/len(imgs)) )
        sys.stdout.flush()
return imgs 


In [ ]:
def get_top_answers(imgs, num_ans):
 """
 Print the questions and returns the time, one answer is repeated
 """
 counts = {}
    for img in imgs:
    try:
    ans = img['answer'].lower() # If the string is a number, it would result into error 27
    except :
    ans = str(img['answer'])
    counts[ans] = counts.get(ans, 0) + 1
    cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
    print('top answer and their counts:')
    print('\n'.join(map(str,cw[:20])))

    vocab = []
    for i in range(min(num_ans,len(cw))):
        vocab.append(cw[i][1])
return vocab[:num_ans] 


In [ ]:
def filter_question(imgs, atoi):
    new_imgs = []
    for i, img in enumerate(imgs):
        new_imgs.append(img)
        print('question number reduce from %d to %d '%(len(imgs), len(new_imgs)))
return new_imgs
manualMap = { 'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three':
 '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7',
 'eight': '8', 'nine': '9', 'ten': '10' }
imgs_train = json.load(open('trainset.json' , 'r')) # Unnnecesarily, i have used the same
file two times
28
num_ans = 1000
top_ans = get_top_answers(imgs_train, num_ans)
atoi = {w:i for i,w in enumerate(top_ans)} # Word : Count
itoa = {i:w for i,w in enumerate(top_ans)} # Count : Word
feat_dim = 300 # 300 Dimensional Vector
imgs_data_train = json.load(open('trainset.json' , 'r')) # trainset.json
num_ans = 10 # Even 1 should work fine, but I had taken reference from COCO
dataset, and hence, 10 (10 represents the top 10 answers to a picture)
method = 'nltk'
max_length = 21 # Max Length of the question
dir_path = "QA" # The path where we will be storing .h5 file
N = len(imgs_data_train)
image_path = 'VQA_RAD Image Folder'

In [ ]:
def save_data():
img_path = image_path+img['image_name']

     for i,img in enumerate(imgs_data_train):
 #print('X' , img['ques_id'])
 
    s = img['question']
    print(i,s) # Print the number and the question
     if method == 'nltk':
    try:
    txt = word_tokenize(str(s).lower())
    except :
    txt = str(s)

    else:
    txt = tokenize(s)
    img['processed_tokens'] = txt
    question_id = img['qid']
    feat = np.array(extract_feat(img['processed_tokens']))
    label_arrays = np.zeros((1, max_length, feat_dim), dtype='float32')
    label_length = min(max_length, len(feat)) # record the length of this sequence
    label_arrays[0, :label_length, :] = feat
    try:
    ans_arrays = atoi[img['answer'].lower()]
    except :
    ans_arrays = atoi[str(img['answer'])]
    f = h5py.File(os.path.join( dir_path , str(question_id) + '.h5'), "w")
    f.create_dataset("ques_train", dtype='float32', data=label_arrays)
    f.create_dataset("answers", dtype='uint32', data=ans_arrays)
    f.close()
return data = save_data() 


In [ ]:
import json
import h5py
import numpy as np
import copy
from random import shuffle, seed
import sys
import os.path
30
import argparse
import glob
import numpy as np
import scipy.io
import pdb
import string
import h5py
import nltk
from nltk.tokenize import word_tokenize
import gensim
import json
import re
import cv2
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
import os
import matplotlib.pyplot as plt
dropout_rate = 0.4
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def extract_feat(doc):
    feat = []
    for word in doc:
    try:
    feat.append(model_w2v[word])

    except:
    pass
 return feat 

In [ ]:
def tokenize(sentence):
    return [i for i in re.split(r"([-.\"',:? !\$#@~()*&\^%;\[\]/\\\+<>\n=])", sentence) if i!=''
and i!=' ' and i!='\n'];
def prepro_question(imgs, method):

 # preprocess all the question
 print('example processed tokens:')
    for i,img in enumerate(imgs):
    s = img['question'].lower()
    if method == 'nltk':
        txt = word_tokenize(str(s).lower())
    else:
        txt = tokenize(s)
    img['processed_tokens'] = txt
    if i < 10: print(txt)
        if i % 1000 == 0:
            sys.stdout.write("processing %d/%d (%.2f%% done) \r" % (i, len(img),
i*100.0/len(imgs)) )
sys.stdout.flush()
return imgs 


In [ ]:
def get_top_answers(imgs, num_ans):
    counts = {}
    for img in imgs:

    try:
    ans = img['answer'].lower()
    except :
    ans = str(img['answer'])
    counts[ans] = counts.get(ans, 0) + 1 # Frequency count
    cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
 # print('top answer and their counts:')
 # print('\n'.join(map(str,cw[:20])))

    vocab = []
    for i in range(min(num_ans,len(cw))):
    vocab.append(cw[i][1])
return vocab[:num_ans]

In [ ]:
def filter_question(imgs, atoi):
    new_imgs = []
    for i, img in enumerate(imgs):
    new_imgs.append(img)
    print('question number reduce from %d to %d '%(len(imgs), len(new_imgs)))
return new_imgs
def image_layer(input_shape):
    base_model = tf.keras.applications.VGG16(input_shape=input_shape,include_top=False,weights='imagenet')
    base_model.trainable = False # Do not train it

    x = base_model.layers[-2].output # Shape would be (28*28*512)
    x = tf.reshape(x , [-1,x.shape[2]*x.shape[1] , x.shape[3]]) # Shape would be (1,784,512)
    x = tf.keras.layers.Dense(1024) # This step can be found out in the slides, that after feature extraction, they are connecting a dense layer, slide - 6 (Transforminto a same size vector)
return x
def vgg_preprocessing(model,image):
return model(image)
def load_data():
    images = []
    questions = []
    answers = []
    ids = []

 #print(start,end)
 #arrs = np.random.randint(0,len(imgs_data_train),batch)
 #data = [imgs_data_train[i] for i in arrs]
data = imgs_data_train # trainset.json
model = image_layer(input_shape = (448,448,3)) # Making VGG16 Model
 for i,img in enumerate(data):

    img_path = img['image_name'] # Image Name
    question_id = img['qid'] # Question id

 #label_arrays = np.zeros((1, max_length, feat_dim), dtype='float32') #
Somethings are taken directly from

 with h5py.File(os.path.join(dir_path,str(question_id) + '.h5'),'r') as hf:
    question = hf['.']['ques_train'][()] # Embedded question
    answer = hf['.']['answers'][()] # Embedded answer

    image = cv2.imread(os.path.join('VQA_RAD Image Folder',img_path) ,
cv2.IMREAD_COLOR) # Reading the image
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
    image = cv2.resize(image , (448,448)) # Reshape
 #image = vgg_preprocessing(model,image)
 # Apply VGG16 Preprocessings
    images.append(image)
    questions.append(np.array(question))
    answers.append(np.array(answer))
    ids.append(question_id)
    if i%100==0:
        print("Processed =>",i,' which is',round(100*i/len(data),2),'%')

questions = np.reshape(np.array(questions) , [-1,max_length,feat_dim])
return (np.array(images) , questions ,np.array(answers) , np.array(ids))
imgs_train = json.load(open('trainset.json' , 'r'))
num_ans = 1000
top_ans = get_top_answers(imgs_train, num_ans )
atoi = {w:i for i,w in enumerate(top_ans)}
itoa = {i:w for i,w in enumerate(top_ans)}
feat_dim = 300
imgs_data_train = json.load(open('trainset.json' , 'r'))
num_ans = 10
method = 'nltk'
max_length = 21
dir_path = "QA"
N = len(imgs_data_train)

In [ ]:
import tensorflow as tf
import numpy as np
import keras,h5py
import pandas as pd
import os
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from keras.applications.vgg16 import VGG16
import keras.activations
import keras.backend as kbe
from keras.callbacks import EarlyStopping
import tensorflow.keras.layers as layers
from keras.layers import Activation, Add, Concatenate, Conv1D, Dense, Dropout,
Embedding, Softmax
from keras.layers import Input, GlobalMaxPooling1D, Lambda, Multiply, RepeatVector,Reshape
from keras.layers import BatchNormalization 

In [ ]:
from keras.models import Model
from keras.regularizers import l2
import pickle
from pprint import pprint
''' These parameters are for some of the previous attempts, so ignore it, the main part is :
datagen = load_data(), and I don't want to remove all of these,
because indeed there can be some ideas coming out from these lines of code'''
embed_size = 300
q_len = 21
height = 224
width = 224
lstm_units = 256
attention_dim = 512
num_output = 1000
max_questions = 3064
batch_size = 32
lr = 0.001
articles = ['a', 'an', 'the']
manualMap = { 'none': '0', 'zero': '00 'one': '1', 'two': '2', 'three':
 '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7',
 'eight': '8', 'nine': '9', 'ten': '10' }
datagen = load_data() # Load the Data
images,questions,answers,ids =datagen[0],datagen[1],datagen[2],datagen[3]

print("Images have a size of:",images.shape)
print("Questions have a size of:",questions.shape)
print("Answers have a size of:",answers.shape)
print("Ids have a size of:",ids.shape)
dir_path = r'QA' # The directory where the .h5 file for each entry is saved
m = 100
for i in range(images.shape[0]):
    ans_array = answers[i]
    image_array = images[i]
    quest_array = questions[i]
    question_id = ids[i]
    f = h5py.File(os.path.join( dir_path , str(question_id) + '.h5'), "w") # Loading the 'h5
file
    f.create_dataset("ques_train", dtype='float32', data=quest_array) # Question
Embedding
    f.create_dataset("image_vector", dtype='float32', data=image_array) # Image
Embedding (Not preprocessed)
    f.create_dataset("answers", dtype='uint32', data=ans_array) # Answers in embedded
form
 f.close()
    if i%m ==0:
        print("Processed =>", i,' total percentage =>', round(100*i/images.shape[0],2),' %')
print("Your processing has been done") 


In [ ]:
import os,h5py
# Image Model
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model

import warnings
warnings.filterwarnings('ignore')
''' Preprocessing with the VGG 16 Model '''
model = tf.keras.applications.VGG16(include_top=False,weights='imagenet',
 input_shape=(448,448,3))
#print("The Last layer")
last_layer = model.layers[-1].output # Last layer has an output layer of (14,14,512)
model = Model(model.input,last_layer)
model.trainable = False
# print(model.summary())
def extract_feature(image):
 ''' Preprocessing with VGG Netowrk'''
 image = model(image)
return image # Shape is (196,512)
''' The below model will convert (196,512) to (21,300) (i.e same as the dimension of word embedding) '''
dimen_red = tf.keras.Sequential() # Use for converting (196,512) -> (21,300)
dimen_red.add(tf.keras.layers.Conv2D(300,kernel_size=(1,1),input_shape= (14,14,512)))
dimen_red.add(tf.keras.layers.Reshape((196,300)))
dimen_red.add(tf.keras.layers.Permute((2,1))) # Reshaping about the axis, useful for
applying the dense network
dimen_red.add(tf.keras.layers.Dense(21))
dimen_red.add(tf.keras.layers.Permute((2,1))) # Reshaping about the axis, useful for
applying the dense network

train_dir = r'QA/' # Containing .h5 file
images = []
ans = []
ques = []
count = 0
content = os.listdir(train_dir)[:2300] # The GPU Memory became full after this, hence
had to take just these much samples :(
length = len(content)
for i in content:
 # Reading the data
 file = h5py.File(train_dir+i)
 images.append(np.array(file['.']['image_vector'][()]))
 ans.append(np.array(file['.']['answers'][()]))
 ques.append(np.array(file['.']['ques_train'][()]))
 count+=1
 if count%100 == 0:
    print("The count is:",count,"and the percentage proportion is:",round(100*count/length,2),'%')
images = tf.convert_to_tensor(np.array(images)) # For the GPU purpose
ans = tf.convert_to_tensor(np.array(ans))
ques = tf.convert_to_tensor(np.array(ques))
l = []
length = images.shape[0]
for i,j in enumerate(images):
    l.append(dimen_red(j)) # Making it to the same shape as that of question embedding
    if i%100 ==0:
       print("The count is:",i,"and the percentage proportion is:",round(100*i/length,2),'%')
images = tf.convert_to_tensor(np.array(l))

images = tf.reshape(images,[length,21,300])
img = images #Tensor containing images
que = ques # Tensor containing question vector
img = img/255.0 # Normalizing
que.shape,img.shape
ques = tf.keras.layers.Input((21,300)) # Input Model (for ques)
images = tf.keras.layers.Input((21,300)) # Input Model (for images)
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D,
SpatialDropout1D,Flatten,Concatenate
''' Imagica is for the preprocessing of the image part'''
imagica = Dense(512,activation='tanh')(images)
imagica = Dense(512,activation='tanh')(images)
''' quesa is for the ques layer, which means preprocessing of the question layer'''
quesa = LSTM(512, dropout = 0.3,return_sequences = True,input_shape =
(21,300))(ques)
quesa = Dense(512, activation = 'relu')(quesa)
quesa = Dropout(0.3)(quesa)
quesa = Dense(512, activation = 'relu')(quesa)
quesa = Dropout(0.3)(quesa)

''' Concatenating both image and the question layer'''
quesa = Concatenate()([quesa,imagica])
quesa = Flatten()(quesa)
out = tf.keras.layers.Dense(476,activation='softmax')(quesa) # Final output has 476
different categories, you can check by finding length of uniquue answers :)
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
answers = tf.keras.utils.to_categorical(ans)
answers.shape
model.summary()
history=model.fit([img,que],ans,epochs = 50,batch_size=32,verbose=1)
model.save('VQA_Model')
prediction = tf.argmax(model.predict([img,que]),axis=1).numpy()
count=0
for i in range(len(ans)):
    if(prediction[i]==ans[i]):
        count=count+1
acc=(count/len(ans))*100
acc
import json
x = open('trainset.json','r')
train = json.load(x)

train[0]
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
import pickle
x = open('cache/trainval_label2ans.pkl','rb')
a2lab = pickle.load(x)
import cv2
img_path = 'VQA_RAD Image Folder/'
import matplotlib.pyplot as plt
%matplotlib inline
for i in range(760,770,2):
    plt.figure(figsize=(10,8))

    image = cv2.imread(img_path+ train[i]['image_name'])
    plt.imshow(image)
    value = "Actual: "+str(train[i]['answer'])+' predicted value: '+ str(a2lab[prediction[i]])
    plt.axis('off')
    print("The Question is:",train[i]['question'])
    print("The answer is:",value)
    print("*"*50)
    plt.show()